# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd

In [ ]:
from sqlalchemy import create_engine

In [ ]:
# load data from database
engine = create_engine('sqlite:///DataBase.db')
df = pd.read_sql('SELECT * FROM df', engine)

In [ ]:
df.head(3)

In [ ]:
df.genre.unique()

#### cleaning non-binary columns:

In [ ]:
df.related.value_counts()

In [ ]:
bad_samples = []
for i in df.index:
    if df.loc[i,'related'] == 2:
        bad_samples.append(i)

In [ ]:
len(bad_samples)

In [ ]:
good_samples = [i for i in df.index if i not in bad_samples ]

In [ ]:
# checking:
len(good_samples)+len(bad_samples), df.shape[0]

In [ ]:
df = df.iloc[good_samples,]
df.head(2)

In [ ]:
# final check:
df.related.value_counts()

#### splitting  feature vs target:

In [ ]:
features = ['message']

In [ ]:
X = df[features]
X.head(3)

In [ ]:
non_target_cols = ['id','message','original','genre'] 
non_target_cols

In [ ]:
target_cols = df.columns[~df.columns.isin(non_target_cols)]
target_cols

In [ ]:
Y = df[target_cols]
Y.head(3)

### 2. Write a tokenization function to process your text data

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import re
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

In [ ]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize) ),
    ('tfidf', TfidfTransformer() ),
    ('clf', MultiOutputClassifier(RandomForestClassifier() ) )
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X.message,Y)

In [ ]:
(X_train.shape , Y_train.shape)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [ ]:
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
clf = pipeline.fit(X_train, Y_train)

In [ ]:
Y_pred = pipeline.predict(X_test)

In [ ]:
(Y_test.shape, Y_pred.shape)

In [ ]:
[type(Y_test), type(Y_pred)]

In [ ]:
Y_pred = pd.DataFrame(Y_pred,columns=Y_test.columns)
Y_pred.head(2)

In [ ]:
import numpy as np

In [ ]:
from sklearn.utils.multiclass import type_of_target

In [ ]:
[type_of_target(Y_test), type_of_target(Y_pred)]

In [ ]:
import inspect
inspect.signature(classification_report)

In [ ]:
print(classification_report(Y_test,Y_pred,target_names=Y_test.columns))

In [ ]:
print(f1_score((Y_test), (Y_pred),average='macro'))

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# parameters in CountVectorizer
inspect.signature(CountVectorizer)

In [ ]:
# parameters in TfidfTransformer
inspect.signature(TfidfTransformer)

In [ ]:
# parameters in RandomForestClassifier
inspect.signature(RandomForestClassifier)

In [ ]:
list(pipeline.get_params().keys())

In [ ]:
parameters = {
    'vect__tokenizer': [tokenize,None], # for count vectorizer
    'tfidf__norm': ['l1','l2'], # for tfidf
    'clf__estimator__criterion': ['gini','entropy'] # for Random Forest
}

# Note: using 'clf__criterion' as syntax generates an error:
# ValueError: Invalid parameter criterion for estimator MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
# By doing cv.get_params().keys() one can see the above one is the right syntax

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
inspect.signature(GridSearchCV)

In [ ]:
cv = GridSearchCV(pipeline, param_grid=parameters)

In [ ]:
cv.fit(X_train,Y_train)

In [ ]:
cv.best_estimator_

In [ ]:
pd.DataFrame(cv.cv_results_)

In [ ]:
cv.best_score_

In [ ]:
cv.best_params_

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
Y_pred_cv = cv.best_estimator_.predict(X_test)

In [ ]:
print(classification_report(Y_test,Y_pred_cv,target_names=Y_test.columns))

In [ ]:
print(f1_score(Y_test,Y_pred,average='macro'))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

##### verb extractor method from class

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
## from Mentor:

class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            except:
                return False
        return False
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
from sklearn.pipeline import FeatureUnion

In [ ]:
pipeline_improved = Pipeline([
    ('features', FeatureUnion([
        ('nlp_pipeline',  Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize) ),
            ('tfidf', TfidfTransformer() )
                                  ]) ),
        ('txt_extract', StartingVerbExtractor() )
                            ]) ),      
        
    ('knn', MultiOutputClassifier(KNeighborsClassifier() ) )
                            ])

In [ ]:
list(pipeline_improved.get_params().keys())

In [ ]:
knn = pipeline_improved.fit(X_train, Y_train)

In [ ]:
knn.score

In [ ]:
for att in dir(knn):
    if not att.startswith('_'):
        print(att)

In [ ]:
Y_pred_improved = pipeline_improved.predict(X_test)

In [ ]:
print(classification_report(Y_test,Y_pred_improved,target_names=Y_test.columns))


In [ ]:
type(report)

In [ ]:
import sklearn
print(sklearn.__version__)

In [ ]:
score = f1_score(Y_test,Y_pred_improved, average='macro')

In [ ]:
score

### Pipeline with two estimators:

In [ ]:
estimators = [('knn', MultiOutputClassifier(KNeighborsClassifier()  )),
              ('clf', MultiOutputClassifier(RandomForestClassifier()))]

In [ ]:
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('nlp_pipeline',  Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize) ),
            ('tfidf', TfidfTransformer() )]) ),
        ('txt_extract', StartingVerbExtractor() )]) ),      
    ('estimators', [('knn', MultiOutputClassifier(KNeighborsClassifier()  )),
                    ('clf', MultiOutputClassifier(RandomForestClassifier()))
                   ]
     )
                    ])

### 9. Export your model as a pickle file

In [ ]:
import pickle

In [ ]:
pickle.dump(knn,open('knn.pkl','wb'))

In [ ]:
pickle.dump(cv.best_estimator_,open('rfc_cv.pkl','wb'))

In [ ]:
pickle.dump(clf,open('clf.pkl','wb'))

In [ ]:
import joblib

In [ ]:
joblib.dump( , "classifier.pkl")

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

## Testing `train_classifier.py` :

In [1]:
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KNeighborsClassifier
import pickle
import joblib

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [23]:
database_filepath, model_filepath = sys.argv[1:]

In [25]:
model_filepath

'/root/.local/share/jupyter/runtime/kernel-ec344199-f04e-4ab8-b530-50f52ddffac0.json'

In [24]:
database_filepath

'-f'

#### load_data()

In [2]:
def load_data(database_filepath):
    """
    Loads data from SQL Database and transforms it for model training
    
    :param database_filepath: SQL database file (string)
    
    :returns x: Features (dataframe)
    :returns y: Targets (dataframe)
    :returns category_names: Target labels (list)
    """
    
    engine = create_engine('sqlite:///{}'.format(database_filepath))
    df = pd.read_sql('SELECT * FROM df', engine)
    
    bad_samples = []
    for i in df.index:
        if df.loc[i,'related'] == 2:
            bad_samples.append(i)
    good_samples = [i for i in df.index if i not in bad_samples ]   
    df = df.iloc[good_samples,]   

    features = ['message']
    X = df[features]
    X = X['message']
    
    non_target_cols = ['id','message','original','genre'] 
    target_cols = df.columns[~df.columns.isin(non_target_cols)]
    Y = df[target_cols]
    
    category_names = Y.columns
    return X,Y, category_names

In [3]:
X, Y, category_names = load_data('DataBase.db')

In [4]:
X.head(3)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
Name: message, dtype: object

In [5]:
Y.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

#### buil_model()

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [9]:
X_test.shape, Y_test.shape

((5206,), (5206, 36))

In [8]:
def tokenize(text):
    stop_words = stopwords.words('english')
    lemmatizer = WordNetLemmatizer()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    
    return tokens

In [10]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            except:
                return False
        return False
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged) 

In [12]:
def build_model():
    """
    Builds a model using Random Forest Classifier. Data is transformed in pipeline using Tokenization, Count Vectorizer,
    Tfidf Transformer and
    
    :return cv: Trained model after performing grid search (GridSearchCV model)
    """
    
    pipeline = Pipeline([
    ('features', FeatureUnion([
        ('nlp_pipeline',  Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize) ),
            ('tfidf', TfidfTransformer() )]) ),
        ('txt_extract', StartingVerbExtractor() )]) ),          
    ('clf', MultiOutputClassifier(RandomForestClassifier() ) )])
    
    parameters = {
    'features__nlp_pipeline__vect__tokenizer': [tokenize,None], # for count vectorizer
    'features__nlp_pipeline__tfidf__norm': ['l1','l2'], # for tfidf
    'clf__estimator__criterion': ['gini','entropy']} # for Random Forest
    
    #cv = GridSearchCV(estimator=pipeline, param_grid=parameters,scoring='f1_macro')
    
    cv = pipeline
    return cv

In [13]:
model = build_model()

In [14]:
model

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

#### evaluate_model():

In [15]:
X_train.shape, Y_train.shape

((20822,), (20822, 36))

In [17]:
model.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('nlp_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [20]:
def evaluate_model(model, X_test, Y_test, category_names):                     
    """
    Measures model's performance on test data and prints out results.
    :param model: trained model (GridSearchCV Object)
    :param X_test: Test features (dataframe)
    :param Y_test: Test targets (dataframe)
    :param category_names: Target labels (dataframe)
    
    :return model_report: Dataframe with Model Performance report
    """             
    Y_pred = model.predict(X_test) #best_estimator_
    Y_pred = pd.DataFrame(Y_pred,columns=Y_test.columns)
    #print("\nBest Parameters:", model.best_params_)
    #score = f1_score( Y_test,Y_pred, average='macro')
    
    return print(classification_report(Y_test,Y_pred,target_names=category_names))

In [21]:
evaluate_model(model, X_test, Y_test, category_names)

                        precision    recall  f1-score   support

               related       0.84      0.92      0.88      3974
               request       0.80      0.45      0.57       887
                 offer       0.00      0.00      0.00        21
           aid_related       0.75      0.60      0.67      2162
          medical_help       0.55      0.09      0.15       410
      medical_products       0.66      0.09      0.15       265
     search_and_rescue       0.33      0.01      0.01       137
              security       0.00      0.00      0.00        86
              military       0.71      0.09      0.16       187
           child_alone       0.00      0.00      0.00         0
                 water       0.82      0.27      0.41       345
                  food       0.86      0.54      0.66       578
               shelter       0.78      0.28      0.41       446
              clothing       0.60      0.08      0.14        78
                 money       0.86      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [30]:
model_filepath = 'classifier.pkl'

In [28]:
def save_model(model, model_filepath):
    """
    Function to save trained model as pickle file.
    :param model: Trained model (GridSearchCV Object)
    :param model_filepath: Filepath to store model (string)
    :return: None
    """
    pickle.dump(model, open(model_filepath, 'wb'))
    #pickle.dump(model,model_filepath)

In [31]:
save_model(model, model_filepath)

In [ ]:
# load data from database
engine = create_engine('sqlite:///DataBase.db')
df = pd.read_sql('SELECT * FROM df', engine)